In [ ]:
import jax
from genjax import ChoiceMapBuilder as C
from genjax import GenericProblem, bernoulli, beta, gen, pretty
from genjax.incremental import Diff, NoChange, UnknownChange
from jax import jit

pretty()


# Define a generative function
@gen
def beta_bernoulli_process(u):
    p = beta(1.0, u) @ "p"
    v = bernoulli(p) @ "v"
    return 2 * v

1) Generate a traced sample

In [ ]:
key = jax.random.PRNGKey(0)
trace = jax.jit(beta_bernoulli_process.simulate)(key, (0.5,))

1.1 Print the return value

In [ ]:
trace.get_retval()

1.2 Print the choice_map, i.e. the list of internal random choices made during the execution

In [ ]:
trace.get_sample()

Alternative way to print the choice_map

In [ ]:
trace.get_choices()

Print specific subparts of the choice_map

In [ ]:
trace.get_sample()["p"]

Then, we can create a choice_map of observations and perform diverse operations on it.
We can set the value of an address in the choice_map

In [ ]:
chm = C["p"].set(0.5) ^ C["v"].set(1)
chm

A different way to achieve the same result

In [ ]:
chm = C["p"].set(0.5).at["v"].set(1)
chm

This also works for hierarchical addresses

In [ ]:
chm = C["p", "v"].set(1)
chm

We can also directly set a value in the choice_map

In [ ]:
chm = C.v(5.0)
chm

We can also create an empty choice_map

In [ ]:
chm = C.n()
chm

2) Compute log probabilities

2.1 Print the log probability of the trace

In [ ]:
trace.get_score()

2.2 Print the log probability of an observation encoded as a ChoiceMap under the model

It returns both the log probability and the return value

In [ ]:
chm = C["p"].set(0.5) ^ C["v"].set(1)
args = (0.5,)
beta_bernoulli_process.assess(chm, args)

Note that the ChoiceMap should be complete, i.e. all random choices should be observed

In [ ]:
chm_2 = C["v"].set(1)
try:
    beta_bernoulli_process.assess(chm_2, args)
except ValueError as e:
    print(e)

3) Generate a sample conditioned on the observations

We can also use a partial ChoiceMap as a constraint/observation and generate a full trace with these constraints.

In [ ]:
key = jax.random.PRNGKey(42)
partial_chm = C["v"].set(1)  # Creates a ChoiceMap of observations
args = (0.5,)
trace, weight = beta_bernoulli_process.importance(
    key, partial_chm, args
)  # Runs importance sampling

This returns a pair containing the new trace and the log probability of produced trace under the model

In [ ]:
trace.get_sample()

In [ ]:
weight

4) Update a trace.

We can also update a trace. This is for instance useful for performance optimizations in Metropolis-Hastings algorithms where often most of the trace doesn't change between time steps.

We first define a model for which changing the argument will force a change in the trace.

In [ ]:
@gen
def beta_bernoulli_process(u):
    p = beta(1.0, u) @ "p"
    v = bernoulli(p) @ "v"
    return 2 * v

We then create an trace to be updated and constraints.

In [ ]:
key = jax.random.PRNGKey(42)
jitted = jit(beta_bernoulli_process.simulate)
old_trace = jitted(key, (0.5,))
constraint = C["v"].set(1)

Now the update uses a form of incremental computation.
It works by tracking the differences between the old new values for arguments.
Just like for differentiation, it can be achieved by providing for each argument a tuple containing the new value and its change compared to the old value.

If there's no change for an argument, the change is set to NoChange.

In [ ]:
arg_diff = (Diff(1.0, NoChange),)

If there's a change, the change is set to the difference between the new and old value.

In [ ]:
arg_diff = (Diff(3.0, 2.0),)

If there's an unknown change, the change is set to UnknownChange.

In [ ]:
arg_diff = (Diff(1.0, UnknownChange),)

We then we bundle together the arguments and the constraint as a `GenericProblem`, a simple instance of an `UpdateProblem`.

In [ ]:
update = GenericProblem(arg_diff, constraint)

We finally use the update method by passing it a key, the trace to be updated, and the update to be performed.

In [ ]:
jitted_update = jit(beta_bernoulli_process.update)

new_trace, weight_diff, ret_diff, discard_choice = jitted_update(key, old_trace, update)

We can compare the old and new values

In [ ]:
old_trace.get_sample()

In [ ]:
print("Old value for p:", old_trace.get_sample()["p"])
print("New value for p:", new_trace.get_sample()["p"])

weight_diff, ret_diff, discard_choice

   5. A few more convenient methods

5.1 `propose`

It uses the same inputs as `simulate` but returns the sample, the score and the return value

In [ ]:
sample, score, retval = jit(beta_bernoulli_process.propose)(key, (0.5,))
sample, score, retval

5.2 `get_gen_fn`

It returns the generative function that produced the trace.

In [ ]:
trace.get_gen_fn()

5.3 `get_args`

It returns the arguments passed to the generative function used to produce the trace

In [ ]:
trace.get_args()

5.4 `get_subtrace`

It takes a `StaticAddress` as argument and returns the sub-trace of a trace rooted at these addresses

In [ ]:
subtrace = trace.get_subtrace(("p",))
subtrace

In [ ]:
subtrace.get_sample()